## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-21-13-03-00 +0000,wsj,GE Aerospace Boosts Guidance as Air-Travel Dem...,https://www.wsj.com/business/earnings/ge-aeros...
1,2025-10-21-13-00-00 +0000,nypost,How ‘Botox’ is helping Ukrainian soldiers who’...,https://nypost.com/2025/10/21/health/how-botox...
2,2025-10-21-12-56-15 +0000,nyt,France Sends Former President Nicolas Sarkozy ...,https://www.nytimes.com/2025/10/21/world/europ...
3,2025-10-21-12-53-58 +0000,bbc,Japan's first female leader: A historic moment...,https://www.bbc.com/news/articles/cze6dgxlzwxo...
4,2025-10-21-12-52-54 +0000,nyt,"Nations Hesitate to Send Troops to Gaza, Feari...",https://www.nytimes.com/2025/10/21/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2392/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
68,trump,48
245,louvre,15
341,new,15
362,will,14
37,gaza,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
205,2025-10-20-22-36-00 +0000,wsj,"President Trump’s $100,000 H-1B visa fee will ...",https://www.wsj.com/politics/policy/u-s-narrow...,126
143,2025-10-21-04-55-46 +0000,nypost,Student loan debt will be forgiven for million...,https://nypost.com/2025/10/21/us-news/student-...,116
257,2025-10-20-20-06-32 +0000,latimes,Trump again suggests sending troops to San Fra...,https://www.latimes.com/california/story/2025-...,113
193,2025-10-20-23-05-53 +0000,nypost,"Trump can deploy National Guard to Portland, O...",https://nypost.com/2025/10/20/us-news/9th-circ...,109
189,2025-10-20-23-45-15 +0000,nyt,Appeals Court Ruling Allows Trump to Deploy Na...,https://www.nytimes.com/2025/10/20/us/appeals-...,106


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
205,126,2025-10-20-22-36-00 +0000,wsj,"President Trump’s $100,000 H-1B visa fee will ...",https://www.wsj.com/politics/policy/u-s-narrow...
25,74,2025-10-21-11-50-06 +0000,nyt,Vance Lands in Israel as U.S. Tries to Shore U...,https://www.nytimes.com/2025/10/21/world/middl...
193,61,2025-10-20-23-05-53 +0000,nypost,"Trump can deploy National Guard to Portland, O...",https://nypost.com/2025/10/20/us-news/9th-circ...
44,56,2025-10-21-11-17-00 +0000,wsj,The Treasury Department instructed employees n...,https://www.wsj.com/politics/policy/treasury-t...
144,53,2025-10-21-04-53-16 +0000,wapo,"Japan elects first female leader, a hawk who p...",https://www.washingtonpost.com/world/2025/10/2...
325,41,2025-10-20-15-10-02 +0000,nypost,French authorities admit they ‘failed’ in Louv...,https://nypost.com/2025/10/20/world-news/louvr...
143,39,2025-10-21-04-55-46 +0000,nypost,Student loan debt will be forgiven for million...,https://nypost.com/2025/10/21/us-news/student-...
33,36,2025-10-21-11-36-00 +0000,wsj,Lockheed Martin Raises Full-Year Profit Outloo...,https://www.wsj.com/business/earnings/lockheed...
201,35,2025-10-20-22-46-37 +0000,nyt,"To Counter China’s Hold on Rare Minerals, Trum...",https://www.nytimes.com/2025/10/20/us/politics...
198,35,2025-10-20-23-00-00 +0000,wsj,Business and political leaders have intensifie...,https://www.wsj.com/politics/elections/nyc-may...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
